# Import Libraries

In [30]:
# run this code to install googleapiclient 
# !pip3 install --upgrade google-api-python-client

from googleapiclient.discovery import build
from IPython.display import JSON

# for data preprocessing
import pandas as pd
import numpy as np

In [31]:
# Youtube API key (change it later)
api_key = 'api_key'

In [32]:
# Channel -> PBS Space Time
channel_ids = ['UC7_gcs09iThXybpVgjHZ_7g']

In [33]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(api_service_name, api_version, developerKey = api_key)

# Gathering the Data of 'PBS SpaceTime' 

In [34]:
# getting all the statistics of this channel

def get_channel_stats(youtube, channel_id):
    
    all_data = []
    request = youtube.channels().list(part="snippet,contentDetails,statistics", 
                                  id= ','.join(channel_ids))
    response = request.execute()
    
    # loop through the items
    for item in response['items']:
        data = {'ChannelName': item['snippet']['title'],
                'Subscribers': item['statistics']['subscriberCount'],
                'Views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
               }
        all_data.append(data)
        
    return (pd.DataFrame(all_data)) 

In [35]:
channel_stats = get_channel_stats(youtube, channel_ids)

In [36]:
channel_stats

,ChannelName,Subscribers,Views,totalVideos,playlistId
0,PBS Space Time,2900000,413837548,357,UU7_gcs09iThXybpVgjHZ_7g


In [37]:
# getting all the video ids

playlist_id = 'UU7_gcs09iThXybpVgjHZ_7g'

def get_video_ids(youtube, playlist_id):
    
    video_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults = 50
    )
    response = request.execute()
    
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
                    part='contentDetails',
                    playlistId = playlist_id,
                    maxResults = 50,
                    pageToken = next_page_token)
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        
    return video_ids

In [38]:
video_ids = get_video_ids(youtube, playlist_id)

In [39]:
len(video_ids)

357

In [40]:
# to get all the video details

def get_video_details(youtube, video_ids):

    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
    
    return pd.DataFrame(all_video_info)


In [41]:
# Get video details
video_df = get_video_details(youtube, video_ids)
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,sWNTsKX5H5M,PBS Space Time,Quantum Galaxies: The Case for Axionic Dark Ma...,PBS Member Stations rely on viewers like you. ...,"[Space, Outer Space, Physics, Astrophysics, Qu...",2023-06-28T20:22:18Z,146496,8828,None,635,PT14M40S,hd,true
1,TbzZIMQC6vk,PBS Space Time,Did AI Prove Our Proton Model WRONG?,PBS Member Stations rely on viewers like you. ...,"[Space, Outer Space, Physics, Astrophysics, Qu...",2023-06-21T20:15:00Z,1100779,42043,None,3056,PT16M57S,hd,true
2,KrRvJUZaswo,PBS Space Time,What Supernova Distance Would Trigger Mass Ext...,Thank you to Brilliant for Supporting PBS. To ...,"[Space, Outer Space, Physics, Astrophysics, Qu...",2023-06-14T20:15:00Z,740251,24249,None,1648,PT16M35S,hd,true
3,kQNqmUZrEOk,PBS Space Time,Can We Move THE SUN?,PBS Member Stations rely on viewers like you. ...,"[Space, Outer Space, Physics, Astrophysics, Qu...",2023-05-31T20:15:01Z,298633,24003,None,1781,PT13M29S,hd,true
4,Ia2GwIpEdk4,PBS Space Time,Is 'Perpetual Motion' Possible with Superfluids?,Thank you to Brilliant for Supporting PBS. To ...,"[Space, Outer Space, Physics, Astrophysics, Qu...",2023-05-24T19:15:00Z,414468,21541,None,1307,PT18M23S,hd,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,W4UfmOHjiJg,PBS Space Time,What Will Destroy Planet Earth?,Tweet at us! @pbsspacetime\nFacebook: facebook...,"[earth, planet, destroy, destruction, mars, as...",2015-03-11T19:18:12Z,1061960,8455,None,1523,PT8M8S,hd,true
353,gJ5KV3rzuag,PBS Space Time,Should We Colonize Venus Instead of Mars?,Sign Up on Patreon to get access to the Space ...,"[gabe perez-giz, pbs digital studios, space ti...",2015-03-04T21:48:46Z,4717294,82026,None,21576,PT7M37S,hd,true
354,QXfhGxZFcVE,PBS Space Time,How Do You Measure the Size of the Universe?,Tweet at us! @pbsspacetime\nFacebook: facebook...,"[pbs spacetime, Veritasium, Vsauce, It's okay ...",2015-02-25T20:29:28Z,929775,13348,None,1097,PT6M37S,hd,true
355,XAU9ofjcx-Y,PBS Space Time,Is It Irrational to Believe in Aliens?,Tweet at us! @pbsspacetime\nFacebook: facebook...,"[space, aliens, do aliens exist?, Veritasium, ...",2015-02-18T20:53:56Z,828229,14151,None,4036,PT7M52S,hd,true


### Converting the data frame to csv 
- For generating a data report using Sweet Viz library to understand the data better and 
- For Data preprocessing and Feature engineering. Check out the **'Youtube Channel Analysis.ipynb'**

In [42]:
video_df = video_df.to_csv('pbs_video_analysis.csv')